随着项目的推进，整体界面通过布局器已经完成，并且也了解了painter的绘图方法，接下来就是最重要的波形绘制了。项目开发中为了加快进度，会对任务进行拆分，因为波形绘制实际上需要2部分组成：数据接收、存储；数据波形绘制。目前需要我们完成的正是这个数据的波形绘制，并且需要适配不同大小的屏幕。但从技术上还是分为两个部分：动态绘制波形；使显示的波形符合标准。

# 知识点

* painter使用
* 信号与槽
* 定时器

# 窗口界面绘制

为了更好地展示波形，我们在目标窗口上搭建一个显示框架。使用之前研究过的QBoxlayout和QWidget界面控件实现。因为界面颜色默认是相同的无法看出布局器的效果，所以通过修改背景色来进行展示。
界面局部参考
![](./image/cloud_monitor.png)

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy
from PyQt5.QtCore import Qt

app = QApplication([])

# 创建最上面的状态栏并配置颜色
widgetMainTitle = QWidget()
widgetMainTitle.setStyleSheet("background-color: rgb(100,100,100)") 

# 创建3个波形界面并配置颜色
widgetWaveEcg = QWidget()
widgetWaveEcg.setStyleSheet("background-color: black") 
widgetWavbeSpo2 = QWidget()
widgetWavbeSpo2.setStyleSheet("background-color: green") 
widgetWaveResp = QWidget()
widgetWaveResp.setStyleSheet("background-color: black") 

# 波形界面布局，三个波形界面采用垂直布局方式
layoutWave = QVBoxLayout()
layoutWave.addWidget(widgetWaveEcg)
layoutWave.addWidget(widgetWavbeSpo2)
layoutWave.addWidget(widgetWaveResp)

# 右侧温度等参数区域界面创建并配置颜色
widgetHr = QWidget()
widgetHr.setStyleSheet("background-color: rgb(100,100,100)") 
widgetSt = QWidget()
widgetSt.setStyleSheet("background-color: rgb(100,100,200)") 
widgetNibp = QWidget()
widgetNibp.setStyleSheet("background-color: rgb(100,200,100)") 
widgetSpo2 = QWidget()
widgetSpo2.setStyleSheet("background-color: rgb(200,100,100)") 
widgetTemp = QWidget()
widgetTemp.setStyleSheet("background-color: rgb(200,200,100)") 
widgetResp = QWidget()
widgetResp.setStyleSheet("background-color: rgb(100,200,200)") 
widgetCo2 = QWidget()
widgetCo2.setStyleSheet("background-color: rgb(200,100,200)") 

# 右侧部分界面布局，用水平布局+垂直布局方式完成
layoutTableLine1 = QHBoxLayout()
layoutTableLine1.addWidget(widgetHr)
layoutTableLine1.addWidget(widgetSt)

layoutTableLine2 = QHBoxLayout()
layoutTableLine2.addWidget(widgetNibp)

layoutTableLine3 = QHBoxLayout()
layoutTableLine3.addWidget(widgetSpo2)
layoutTableLine3.addWidget(widgetTemp)

layoutTableLine4 = QHBoxLayout()
layoutTableLine4.addWidget(widgetResp)
layoutTableLine4.addWidget(widgetCo2)

layoutTable = QVBoxLayout()
layoutTable.addLayout(layoutTableLine1)
layoutTable.addLayout(layoutTableLine2)
layoutTable.addLayout(layoutTableLine3)
layoutTable.addLayout(layoutTableLine4)

# 波形区域和参数显示区域水平布局
layoutDown = QHBoxLayout()
layoutDown.addLayout(layoutWave)
layoutDown.addLayout(layoutTable)
layoutDown.setStretch(0,2)
layoutDown.setStretch(1,1)

# 主界面布局，采用1:10比例进行布局
layoutMain = QVBoxLayout()
layoutMain.addWidget(widgetMainTitle)
layoutMain.addLayout(layoutDown)
layoutMain.setStretch(0,1)
layoutMain.setStretch(1,10)

# 主界面创建，并使用主界面布局器
widgetMain = QWidget()
widgetMain.resize(800,480)
widgetMain.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
widgetMain.setLayout(layoutMain)
widgetMain.show()

app.exec_()

![](./image/qt_painter_wave_layout.png)

# 画正弦波

为了让问题变得简单，我们把其中一个波形界面拿出来进行研究。并且把这个界面封装成类使等容易使用。

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtCore import Qt

app = QApplication([])

新创建的类型设计为QWidget子类，因为新建类本质上还是一个界面。界面上创建一个标签用于显示显示"ECG"标题，然后再创建一个QWidget界面，这个界面就是最终的画图界面了，我们可以在类中实现一些绘图相关函数。这里可以暂时选择标题与波形界面比例为1:5

In [ ]:
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidget()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)

In [ ]:
wave = EcgWave()
wave.resize(600, 150)
wave.show()

app.exec_()

用新创建的类创建对象并显示，就可以得到我们的波形界面了：
![](./image/qt_ecg_wave_class.png)

至此，画图的场地就已经准备就绪了，然后就需要开始使用painter画图了。正弦波之前，我们先画2个连续的三角形。绘图需要创建绘图类，这里使用qt_painter教程中设计的三角形绘图类进行改进，同时用这个类替换ecgWave中的绘图区域。

In [ ]:
# step 1
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen
from PyQt5.QtCore import QRectF, QPoint, Qt

app = QApplication([])

In [ ]:
# Step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
        painter.setBrush(QColor("#F40002"))
        painter.setPen(QColor("#F40002"))
        self.drawTriangle(painter)
        painter.resetTransform() # 坐标系复位
        painter.end()
    
    def drawTriangle(self, painter):
        height = self.height()
        points = [QPoint(0, height), QPoint(50, height/2), QPoint(100, height)]
        painter.drawPolygon(QPolygon(points), 0)

EcgWave类中只需要更改：
```
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        #self.waveWin = QWidget()
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
```

In [ ]:
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)

In [ ]:
# step 4
wave = EcgWave()
wave.resize(600, 150)
wave.show()

app.exec_()

结果画出来是是多边形,所以会有填充。而曲线的绘制实现有多种方法：
* drawLine/drawLines：绘制线段
* drawPoint/drawPoints：绘制点
* drawPath：绘制路径
这里暂时选择绘点的方式来实现。方法就是水平方向作为时间轴，垂直方向为正弦波的当前值。还是先用三角形来进行验证

重新实现drawTriangle函数，并且把画笔的配置也放在此函数中，修改画笔为2个像素默认是1个像素线条很不清晰，代码如下：

In [ ]:
# step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
#         painter.setBrush(QColor("#F40002"))
#         painter.setPen(QColor("#F40002"))
        self.drawTriangle(painter)
        painter.resetTransform() # 坐标系复位
        painter.end()
    
    def drawTriangle(self, painter):
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)
        
        # draw triangle wave
        height = self.height()
        width = self.width()
        y = height
        dir = 0 # y的计数方向
        for x in range(width):
            if dir == 0:
                y = y - 1
            else:
                y = y + 1
            if y < height/2:
                dir = 1;
            if y == height:
                dir = 0
            painter.drawPoint(x,y) # 假定三角形底角为45

重新执行代码step1~step4即可得到一个三角波界面。
![](./image/qt_wave_triangle.png)
如果想要实现正弦波就比较简单了，只需要把三角波生成并绘图部分替换为正弦波即可。

In [ ]:
import math

# step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
#         self.drawTriangle(painter)
        self.drawSineWave(painter)
        painter.resetTransform() # 坐标系复位
        painter.end()

    def drawTriangle(self, painter):
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)
        
        # draw triangle wave
        height = self.height()
        width = self.width()
        y = height
        dir = 0 # y的计数方向
        for x in range(width):
            if dir == 0:
                y = y - 1
            else:
                y = y + 1
            if y < height/2:
                dir = 1;
            if y == height:
                dir = 0
            painter.drawPoint(x,y) # 假定三角形底角为45
        
    def drawSineWave(self, painter):
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)

        # draw sine wave
        height = self.height()
        width = self.width()

        R = math.pi/100 # 1/2个正弦波周期分为100份
        A  = height/3   # 正弦波幅度取界面高度的1/3
        for x in range(width):
            painter.drawPoint(x, height - A + A * math.sin(R * x)) # 假定三角形底角为45

![](./image/qt_wave_sine.png)

# 使正弦波动态刷新

要想让正弦波动起来就需要不停的刷新屏幕了，这时就需要用到定时器了，需要定时执行刷新操作。当然了这里我们还可以选择另一个方法就是定时器事件，有点像`paintEvent`。这是两种不同的机制：前者是信号与槽机制，后者是事件机制。

举个例子感受下两者的区别：接力赛赛场上，发令者对选手们说：“大家听到枪响，就跑，后面的选手等接到接力棒后开始跑”（真实赛场不是这样的）。前者是选手需要聆听，听到枪响的声音后开始跑，这声音就是信号，选手的跑可等价为槽函数，发令者说的话就像声音与选手之间的信号槽，选手不断监听声音的这种模式即为“观察者模式”。接到接力棒再开始包，换句话说就是接力棒到达这个事件发生后再跑，有点像在等红绿灯，就是需要某个事件发生时才会执行某个动作或某个函数。

有些需求场景下信号与槽和事件均可实现，有些场景两者都会用到。例如鼠标单击事件，按下鼠标时会产生一个事件，然后在事件函数中可以发出信号，进而触发槽函数的调用。

qt中事件使用了一个事件队列来维护，如果事件发生了就会追加到队列尾部，事件的便利是在app.exec_()中执行的，他会不停地遍历事件列表并处理最先发生的事件处理函数，并且会根据事件处理函数的返回值判断事件是否已得到处理，如果返回为False，则逐级向上报送直到事件被处理。并且事件是逐个处理的。

信号没有这么麻烦，qt内部维护着信号与槽函数的连接关系，并在信号产生后调用相关联的槽函数。并且信号产生后qt中在此信号上注册的槽函数都会被调用，如果槽函数中再次产生一个信号，那么这个信号的槽函数会被立即执行。【怎么验证下？】

* 事件是会被传递的，由子控件传递给父控件。信号是会级联的，即信号处理函数（槽函数）中产生信的信号，进而调用新的槽函数。
* 事件是通过队列被处理的，属于异步操作。信号是立即调用槽函数的，属于同步操作。
* 事件的发出者一般是窗口系统，例如鼠标移动、单击等，少数来自系统内部，例如定时器等。信号的发出者是对象。
* 事件处理函数的返回值为True说明事件得到处理，否则会逐级上报。信号的槽函数返回值无意义

这里我们使用事件机制来实现波形的动态刷新。当然如果需要事件响应速度慢导致波形异常时可尝试信号机制。

同样是定时器，针对槽机制有类QTimer，针对事件机制有QBasicTimer。我们在`QWidgetDraw`类中加入定时器`QBasicTimer`, 并重写`timerEvent`函数:
```
...
        self.timer = QBasicTimer()
        self.timer.start(50, self) ## 50ms周期
    
    def timerEvent(self):
        self.update()
```
这里定时器事件处理函数中并没有直接处理波形更新，而是调用了`update`函数，这个函数会发出paintEvent事件，进而使的`paintEvent`函数被调用。

In [ ]:
# step 1
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen
from PyQt5.QtCore import QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])


In [ ]:
import math

# step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(50, self)
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.range = 0 # 圆周角度
    
    def timerEvent(self, event):
        self.update()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
        self.drawSineWave(painter)
        painter.resetTransform() # 坐标系复位
        painter.end()
    
    def drawSineWave(self, painter):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)

        # 计算绘图参数
        height = self.height()
        width = self.width()
        R = math.pi/100 # 1/2个正弦波周期分为100份
        A  = height/3   # 正弦波幅度取界面高度的1/3
        
        painter.drawPoint(self.index, height - A + A * math.sin(self.range))
        
        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 1
        if self.index > width:
            self.index = 0
        self.range = self.range + R

In [ ]:

# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)

In [ ]:
# step 4
wave = EcgWave()
wave.resize(600, 150)
wave.show()

app.exec_()

运行出界面的时候，居然没找到波形，仔细一看原来波形只有1个点...
![](./image/qt_wave_sine_point.png)

原来每次执行paintEvent之前会自动先擦除界面，然后再重新绘制。那么怎么让图图像保持呢？查资料发现以前版本可以通过设置`WRepaintNoErase`参数，可是这个是在QT3中支持的，到QT4就已经不支持了，官方提供的方法是：1. 部分刷新； 2. 在内存中绘图，然后使用painter绘制到界面上。

这里我们选择第二种方法，因为局部刷新需要不断处理刷新区域，会让程序变得很麻烦。而在内存中绘图我们可以使用qpainter的drawPixmap功能，首先在pixmap上绘制图像并使图像保持，然后再用paintEvent将这个画布画出来。

所以，对`QWidgetDraw`进行优化如下代码。

In [ ]:
import math
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import Qt

# step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(50, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.range = 0 # 圆周角度
    
    def timerEvent(self, event):
        self.drawWaveToPixmap()
        self.update()
        
    def drawWaveToPixmap(self):
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawSineWave(pixPainter)
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
#         self.drawSineWave(painter)
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
    
    def drawSineWave(self, painter):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)

        # 计算绘图参数
        height = self.height()
        width = self.width()
        R = math.pi/100 # 1/2个正弦波周期分为100份
        A  = height/3   # 正弦波幅度取界面高度的1/3
        
        painter.drawPoint(self.index, height - A + A * math.sin(self.range))
        
        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 1
        if self.index > width:
            self.index = 0
        self.range = self.range + R

代码的思路很简单，原来的`drawSineWave`是在QWidget上绘图的，现在先在pixmap上绘图，然后再把pixmap画到widget上。

`drawWaveToPixmap`所做的事情就是原来在paintEvent中做的，现在放到了定时器中。因为pixmap会保持绘图内容，所以在paintevent中只需执行`painter.drawPixmap(0, 0, self.pixmap)`即可。

然而pixmap会一直保持数据，程序运行一段时间后变成了这样。
![](./image/qt_wave_sine_move_bug.png)

原因很简单pixmap保持了每一次绘制的图像，明白了原因解决办法也就不难了，写入每个像素点的时候先把当前列清空即可。需要修改`drawSineWave`函数，在执行drawPoint之前先把当前列写为背景色。
```
    def drawSineWave(self, painter):
        # 配置painter画笔参数
        ...
        
        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        painter.drawLine(self.index, 0, self.index, self.height())
        painter.reload() # 还原当前配置

        # 计算绘图参数
        height = self.height()
        width = self.width()
        R = math.pi/100 # 1/2个正弦波周期分为100份
        A  = height/3   # 正弦波幅度取界面高度的1/3
        
        painter.drawPoint(self.index, height - A + A * math.sin(self.range))
        
        # 更新绘图参数，水平方向循环绘图
        ....
```

In [ ]:
# step 1
import math
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

# step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(10, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.range = 0 # 圆周角度
    
    def timerEvent(self, event):
        self.drawWaveToPixmap()
        self.update()
        
    def drawWaveToPixmap(self):
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawSineWave(pixPainter)
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
#         self.drawSineWave(painter)
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
    
    def drawSineWave(self, painter):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)

        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        painter.drawLine(self.index, 0, self.index, self.height())
        painter.restore() # 还原当前配置
        
        # 计算绘图参数
        height = self.height()
        width = self.width()
        R = math.pi/100 # 1/2个正弦波周期分为100份
        A  = height/3   # 正弦波幅度取界面高度的1/3
        
        painter.drawPoint(self.index, height - A + A * math.sin(self.range))
        
        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 1
        if self.index > width:
            self.index = 0
        self.range = self.range + R
        
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)
        
# step 4
wave = EcgWave()
wave.resize(600, 150)
wave.show()

app.exec_()

![](./image/qt_wave_sine_move_done.png)
至此一个动态的正弦波界面就完成了。

如果把正弦波计算的数据替换为心电波形数据就可以显示心电图了。

# 心电波形绘制

```
...
2000, 2000, 2000, 2000, 2000, 2000, 2000, 1992, 1984, 1976,
1968, 1960, 1952, 1944, 1936, 1944, 1952, 2016, 2080, 2136,
2192, 2256, 2320, 2376, 2432, 2488, 2544, 2568, 2592, 2536,
2480, 2424, 2368, 2304, 2240, 2184, 2128, 2072, 2016, 1968,
1920, 1928, 1936, 1944, 1952, 1960, 1968, 1984, 2000,
...
```
这时一段解析后的心电波形数据，我们以此来代替正弦波数据进行显示。数据中的2000对应中心线，上下范围各2000，实际上应该是2048。所以我们要做的事情是把0~4096的数据显示在0~95的范围内。并且我们拿到的一个周期的数据有很多。
之前有做分析过心电值可作为垂直方向坐标，可以用4096除以显示界面垂直方向的高度，得到一个像素点对应的值。
我们根据`drawSineWave`实现`drawEcgWave`函数

In [1]:
# step 1
import math
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

# step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(10, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.ecgDataIndex = 0
    
    def timerEvent(self, event):
        self.drawWaveToPixmap()
        self.update()
        
    def drawWaveToPixmap(self):
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawEcgWave(pixPainter)
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
#         self.drawSineWave(painter)
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
    
    ecgWave = [ 
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2008, 2016, 2016, 2016, 2024, 2032, 2048,
        2064, 2064, 2064, 2072, 2080, 2080, 2080, 2088, 2096, 2104,
        2112, 2112, 2112, 2112, 2112, 2112, 2104, 2096, 2088,
        2080, 2080, 2080, 2072, 2064, 2064, 2064, 2048, 2032, 2032,
        2032, 2016, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 1992, 1984, 1976,
        1968, 1960, 1952, 1944, 1936, 1944, 1952, 2016, 2080, 2136,
        2192, 2256, 2320, 2376, 2432, 2488, 2544, 2568, 2592, 2536,
        2480, 2424, 2368, 2304, 2240, 2184, 2128, 2072, 2016, 1968,
        1920, 1928, 1936, 1944, 1952, 1960, 1968, 1984, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2008, 2016, 2024, 2032, 2032,
        2032, 2048, 2064, 2064, 2064, 2072, 2080, 2088, 2096, 2104,
        2112, 2112, 2112, 2120, 2128, 2136, 2144, 2152, 2160, 2160,
        2160, 2160, 2160, 2168, 2176, 2176, 2176, 2184, 2192,
        2192, 2192, 2192, 2200, 2208, 2208, 2208, 2208, 2208, 2208,
        2208, 2200, 2192, 2192, 2192, 2184, 2176, 2176, 2176, 2168,
        2160, 2160, 2160, 2144, 2128, 2128, 2128, 2128, 2128, 2112,
        2096, 2088, 2080, 2072, 2064, 2064, 2064, 2048, 2032, 2024,
        2016, 2016, 2016, 2008, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000] 
    
    def drawEcgWave(self, painter):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)

        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        painter.drawLine(self.index, 0, self.index, self.height())
        painter.restore() # 还原当前配置
        
        # 绘图
        painter.drawPoint(self.index, self.height()/4096 * self.ecgWave[self.ecgDataIndex])
        
        # 更新参数
        self.index = self.index + 1
        if self.index >= self.width():
            self.index = 0
        
        self.ecgDataIndex = self.ecgDataIndex + 1
        if self.ecgDataIndex >= len(self.ecgWave):
            self.ecgDataIndex = 0

                          
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)
        
# step 4
wave = EcgWave()
wave.resize(600, 150)
wave.show()

app.exec_()

0

![](./image/qt_wave_ecg2_1.png)

# 抽样与放大

针对第一个问题，可以采用抽样的方法解决因为定时器周期是10ms，所以每秒钟有100个数据，而原始数据有500个，所以最简单的方法是每隔5个数据取1个。对应程序，只需要修改：
```
# self.ecgDataIndex = self.ecgDataIndex + 1
self.ecgDataIndex = self.ecgDataIndex + 5
```

In [ ]:
# step 1
import math
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

# step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(10, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.ecgDataIndex = 0
    
    def timerEvent(self, event):
        self.drawWaveToPixmap()
        self.update()
        
    def drawWaveToPixmap(self):
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawEcgWave(pixPainter)
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
#         self.drawSineWave(painter)
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
    
    ecgWave = [ 
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2008, 2016, 2016, 2016, 2024, 2032, 2048,
        2064, 2064, 2064, 2072, 2080, 2080, 2080, 2088, 2096, 2104,
        2112, 2112, 2112, 2112, 2112, 2112, 2104, 2096, 2088,
        2080, 2080, 2080, 2072, 2064, 2064, 2064, 2048, 2032, 2032,
        2032, 2016, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 1992, 1984, 1976,
        1968, 1960, 1952, 1944, 1936, 1944, 1952, 2016, 2080, 2136,
        2192, 2256, 2320, 2376, 2432, 2488, 2544, 2568, 2592, 2536,
        2480, 2424, 2368, 2304, 2240, 2184, 2128, 2072, 2016, 1968,
        1920, 1928, 1936, 1944, 1952, 1960, 1968, 1984, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2008, 2016, 2024, 2032, 2032,
        2032, 2048, 2064, 2064, 2064, 2072, 2080, 2088, 2096, 2104,
        2112, 2112, 2112, 2120, 2128, 2136, 2144, 2152, 2160, 2160,
        2160, 2160, 2160, 2168, 2176, 2176, 2176, 2184, 2192,
        2192, 2192, 2192, 2200, 2208, 2208, 2208, 2208, 2208, 2208,
        2208, 2200, 2192, 2192, 2192, 2184, 2176, 2176, 2176, 2168,
        2160, 2160, 2160, 2144, 2128, 2128, 2128, 2128, 2128, 2112,
        2096, 2088, 2080, 2072, 2064, 2064, 2064, 2048, 2032, 2024,
        2016, 2016, 2016, 2008, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000] 
    
    def drawEcgWave(self, painter):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)

        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        painter.drawLine(self.index, 0, self.index, self.height())
        painter.restore() # 还原当前配置
        
        # 绘图
        painter.drawPoint(self.index, self.height()/4096 * self.ecgWave[self.ecgDataIndex])
        
        # 更新参数
        self.index = self.index + 1
        if self.index >= self.width():
            self.index = 0
        
        self.ecgDataIndex = self.ecgDataIndex + 5
        if self.ecgDataIndex >= len(self.ecgWave):
            self.ecgDataIndex = 0

                          
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)
        
# step 4
wave = EcgWave()
wave.resize(600, 150)
wave.show()

app.exec_()

![](./image/![image.png](attachment:image.png))
可以看到波形明显有所改善。

然后是信号的放大，已知信号是以2048为中心的，并且根据有效数据可以看出数据波动范围在1900~2500之间。这里我们可以设计一个简单的计算：
1. 信号减去2048，此时信号会有正值，会有负值
2. 信号以峰峰值为1200进行归一化处理，即信号除以600
3. 信号乘以显示界面高度的1/2，进行信号放大
4. 信号加上界面高度的1/2，进行信号平移

In [ ]:
# step 1
import math
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

# step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(10, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.ecgDataIndex = 0
    
    def timerEvent(self, event):
        self.drawWaveToPixmap()
        self.update()
        
    def drawWaveToPixmap(self):
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawEcgWave(pixPainter)
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
#         self.drawSineWave(painter)
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
    
    ecgWave = [ 
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2008, 2016, 2016, 2016, 2024, 2032, 2048,
        2064, 2064, 2064, 2072, 2080, 2080, 2080, 2088, 2096, 2104,
        2112, 2112, 2112, 2112, 2112, 2112, 2104, 2096, 2088,
        2080, 2080, 2080, 2072, 2064, 2064, 2064, 2048, 2032, 2032,
        2032, 2016, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 1992, 1984, 1976,
        1968, 1960, 1952, 1944, 1936, 1944, 1952, 2016, 2080, 2136,
        2192, 2256, 2320, 2376, 2432, 2488, 2544, 2568, 2592, 2536,
        2480, 2424, 2368, 2304, 2240, 2184, 2128, 2072, 2016, 1968,
        1920, 1928, 1936, 1944, 1952, 1960, 1968, 1984, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2008, 2016, 2024, 2032, 2032,
        2032, 2048, 2064, 2064, 2064, 2072, 2080, 2088, 2096, 2104,
        2112, 2112, 2112, 2120, 2128, 2136, 2144, 2152, 2160, 2160,
        2160, 2160, 2160, 2168, 2176, 2176, 2176, 2184, 2192,
        2192, 2192, 2192, 2200, 2208, 2208, 2208, 2208, 2208, 2208,
        2208, 2200, 2192, 2192, 2192, 2184, 2176, 2176, 2176, 2168,
        2160, 2160, 2160, 2144, 2128, 2128, 2128, 2128, 2128, 2112,
        2096, 2088, 2080, 2072, 2064, 2064, 2064, 2048, 2032, 2024,
        2016, 2016, 2016, 2008, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000] 
    
    def drawEcgWave(self, painter):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)

        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        painter.drawLine(self.index, 0, self.index, self.height())
        painter.restore() # 还原当前配置
        
        # 取值归一化，放大
        y = round(self.height()/2 - ((self.ecgWave[self.ecgDataIndex] - 2048)/600) * self.height()/2) # 说好的加法怎么变成了减法？
        # 绘图
        painter.drawPoint(self.index, y)
        
        # 更新参数
        self.index = self.index + 1
        if self.index >= self.width():
            self.index = 0
        
        self.ecgDataIndex = self.ecgDataIndex + 5
        if self.ecgDataIndex >= len(self.ecgWave):
            self.ecgDataIndex = 0

                          
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)
        
# step 4
wave = EcgWave()
wave.resize(600, 150)
wave.show()

app.exec_()

![](./image/qt_wave_ecg2_3.png)

数据抽样之后就能明细看到波形的不连续了，这是因为我们是按点描绘的。所以需要移植为直线连接。

核心代码如下，需要分别确定直线的开始位置和结束位置
```
        # 计算绘图参数
        height = self.height()

        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 1
        if self.index > self.width():
            self.index = 1

        lineStart = QPoint()
        lineEnd = QPoint()
        
        # 设置起始、结束点的x坐标
        lineStart.setX(self.index-1)
        lineEnd.setX(self.index)
        # 设置起始点的y坐标
        lineStart.setY(round(self.height()/2 - ((self.ecgWave[self.ecgDataIndex] - 2048)/600) * self.height()/2))
        # 更新参数循环取数
        self.ecgDataIndex = self.ecgDataIndex + 5
        if self.ecgDataIndex >= len(self.ecgWave):
            self.ecgDataIndex = 0
        # 设置结束点的y坐标
        lineEnd.setY(round(self.height()/2 - ((self.ecgWave[self.ecgDataIndex] - 2048)/600) * self.height()/2))
        
        # 直线绘制
        painter.drawLine(lineStart, lineEnd)
```        

In [1]:
# step 1
import math
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

# step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(10, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.ecgDataIndex = 0
    
    def timerEvent(self, event):
        self.drawWaveToPixmap()
        self.update()
        
    def drawWaveToPixmap(self):
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawEcgWave(pixPainter)
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
#         self.drawSineWave(painter)
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
    
    ecgWave = [ 
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2008, 2016, 2016, 2016, 2024, 2032, 2048,
        2064, 2064, 2064, 2072, 2080, 2080, 2080, 2088, 2096, 2104,
        2112, 2112, 2112, 2112, 2112, 2112, 2104, 2096, 2088,
        2080, 2080, 2080, 2072, 2064, 2064, 2064, 2048, 2032, 2032,
        2032, 2016, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 1992, 1984, 1976,
        1968, 1960, 1952, 1944, 1936, 1944, 1952, 2016, 2080, 2136,
        2192, 2256, 2320, 2376, 2432, 2488, 2544, 2568, 2592, 2536,
        2480, 2424, 2368, 2304, 2240, 2184, 2128, 2072, 2016, 1968,
        1920, 1928, 1936, 1944, 1952, 1960, 1968, 1984, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2008, 2016, 2024, 2032, 2032,
        2032, 2048, 2064, 2064, 2064, 2072, 2080, 2088, 2096, 2104,
        2112, 2112, 2112, 2120, 2128, 2136, 2144, 2152, 2160, 2160,
        2160, 2160, 2160, 2168, 2176, 2176, 2176, 2184, 2192,
        2192, 2192, 2192, 2200, 2208, 2208, 2208, 2208, 2208, 2208,
        2208, 2200, 2192, 2192, 2192, 2184, 2176, 2176, 2176, 2168,
        2160, 2160, 2160, 2144, 2128, 2128, 2128, 2128, 2128, 2112,
        2096, 2088, 2080, 2072, 2064, 2064, 2064, 2048, 2032, 2024,
        2016, 2016, 2016, 2008, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000] 
    def drawEcgWave(self, painter):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)

        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        painter.drawLine(self.index, 0, self.index, self.height())
        painter.restore() # 还原当前配置
        
        # 计算绘图参数
        height = self.height()

        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 1
        if self.index > self.width():
            self.index = 1

        lineStart = QPoint()
        lineEnd = QPoint()
        
        # 设置起始、结束点的x坐标
        lineStart.setX(self.index-1)
        lineEnd.setX(self.index)
        # 设置起始点的y坐标
        lineStart.setY(round(self.height()/2 - ((self.ecgWave[self.ecgDataIndex] - 2048)/600) * self.height()/2))
        # 更新参数循环取数
        self.ecgDataIndex = self.ecgDataIndex + 5
        if self.ecgDataIndex >= len(self.ecgWave):
            self.ecgDataIndex = 0
        # 设置结束点的y坐标
        lineEnd.setY(round(self.height()/2 - ((self.ecgWave[self.ecgDataIndex] - 2048)/600) * self.height()/2))
        
        # 直线绘制
        painter.drawLine(lineStart, lineEnd)

                          
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)
        
# step 4
wave = EcgWave()
wave.resize(600, 150)
wave.show()

app.exec_()

0

![](./image/qt_wave_ecg2_4.png)

看样子上面的直线没画好。代码中x方向两点间隔只有1个像素，当垂直方向变化较大时，这条我们不能确定qt会如何绘制这条直线。所以我们进一步增加从波形数据取数的间隔，同时x方向的步进也改为2。
```
        painter.drawLine(self.index, 0, self.index+1, self.height())
        ...
        
        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 2
        if self.index > self.width():
            self.index = 2

        ...
        # 更新参数循环取数
        self.ecgDataIndex = self.ecgDataIndex + 10
        if self.ecgDataIndex >= len(self.ecgWave):
            self.ecgDataIndex = 0
        ...
```

In [ ]:
# step 1
import math
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

# step 2
class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(10, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.ecgDataIndex = 0
    
    def timerEvent(self, event):
        self.drawWaveToPixmap()
        self.update()
        
    def drawWaveToPixmap(self):
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawEcgWave(pixPainter)
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
#         self.drawSineWave(painter)
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
    
    ecgWave = [ 
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2008, 2016, 2016, 2016, 2024, 2032, 2048,
        2064, 2064, 2064, 2072, 2080, 2080, 2080, 2088, 2096, 2104,
        2112, 2112, 2112, 2112, 2112, 2112, 2104, 2096, 2088,
        2080, 2080, 2080, 2072, 2064, 2064, 2064, 2048, 2032, 2032,
        2032, 2016, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 1992, 1984, 1976,
        1968, 1960, 1952, 1944, 1936, 1944, 1952, 2016, 2080, 2136,
        2192, 2256, 2320, 2376, 2432, 2488, 2544, 2568, 2592, 2536,
        2480, 2424, 2368, 2304, 2240, 2184, 2128, 2072, 2016, 1968,
        1920, 1928, 1936, 1944, 1952, 1960, 1968, 1984, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2008, 2016, 2024, 2032, 2032,
        2032, 2048, 2064, 2064, 2064, 2072, 2080, 2088, 2096, 2104,
        2112, 2112, 2112, 2120, 2128, 2136, 2144, 2152, 2160, 2160,
        2160, 2160, 2160, 2168, 2176, 2176, 2176, 2184, 2192,
        2192, 2192, 2192, 2200, 2208, 2208, 2208, 2208, 2208, 2208,
        2208, 2200, 2192, 2192, 2192, 2184, 2176, 2176, 2176, 2168,
        2160, 2160, 2160, 2144, 2128, 2128, 2128, 2128, 2128, 2112,
        2096, 2088, 2080, 2072, 2064, 2064, 2064, 2048, 2032, 2024,
        2016, 2016, 2016, 2008, 2000, 2000, 2000, 2000, 2000,
        2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000] 
    def drawEcgWave(self, painter):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)
        
        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        painter.drawLine(self.index, 0, self.index+1, self.height())
        painter.restore() # 还原当前配置
        
        # 计算绘图参数
        height = self.height()

        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 2
        if self.index > self.width():
            self.index = 2

        lineStart = QPoint()
        lineEnd = QPoint()
        
        # 设置起始、结束点的x坐标
        lineStart.setX(self.index-2)
        lineEnd.setX(self.index)
        # 设置起始点的y坐标
        lineStart.setY(round(self.height()/2 - ((self.ecgWave[self.ecgDataIndex] - 2048)/600) * self.height()/2))
        # 更新参数循环取数
        self.ecgDataIndex = self.ecgDataIndex + 10
        if self.ecgDataIndex >= len(self.ecgWave):
            self.ecgDataIndex = 0
        # 设置结束点的y坐标
        lineEnd.setY(round(self.height()/2 - ((self.ecgWave[self.ecgDataIndex] - 2048)/600) * self.height()/2))
        
        # 直线绘制
        painter.drawLine(lineStart, lineEnd)

                          
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)
        
# step 4
wave = EcgWave()
wave.resize(600, 150)
wave.show()

app.exec_()

![](./image/qt_wave_ecg2_4.png)

至此感觉好像对了。
其实没有，当我高兴地提交任务的时候，领导说：“波形画的不错，你用的是标准波形吗？”

“标准波形？”都没想过，我只是想办法让波形显示的尽量好看。

领导：“心电图绝不是为了好看的，而是有着严格标准的，你去查以下吧。”


# ECG标准

工作没到位，那么就去研究下心电图吧，当然研究也只是研究下波形的一些标准，而不会去研究如何从心电图上看出有什么病。

经过了解：
* 心电波形显示窗口最小30mm
* 心电信号的1mV在屏幕上显示出来应等于10mm，即标准化：1mV=10mm,也有半标准化：1mV=5mm， 双倍标准化：1mV=20mm
* 心电信号走纸速度通常为25mm/s，也有50mm/s和12.5mm/s
* 心电波形一般显示为绿色


看样子主要是一些数学问题。应该不会太难。

Do it